In [62]:
from keras.applications.inception_v3 import InceptionV3

from utils import *
path = './data/'

from keras.callbacks import ModelCheckpoint

## Prep Data

In [59]:
#Now for future iterations, just load the array
val_batches = get_batches(path + 'valid/', batch_size=32, target_size=(299,299))
tst_batches = get_batches(path + 'test_stg1/', batch_size=32, target_size=(299,299))

aug_gen = image.ImageDataGenerator(rotation_range=15, 
                                   width_shift_range=.2, 
                                   height_shift_range=.2, 
                                   zoom_range=.5,
                                  channel_shift_range=.5,
                                  horizontal_flip=True,
                                  vertical_flip=True)

tr_batches = get_batches(path + 'train/', batch_size=32, 
                         gen=aug_gen, target_size=(299,299))


Found 1146 images belonging to 8 classes.
Found 1000 images belonging to 1 classes.
Found 2631 images belonging to 8 classes.


## Fine Tune the Inception V3 Model 
- [Source Code for Inception](https://github.com/fchollet/keras/blob/master/keras/applications/inception_v3.py)
- Skipping the final layer
- Make the layers untrainable
- Add in the fully connected block again
- Train for a few epochs
- Eval

In [64]:
base_model = InceptionV3(weights='imagenet', include_top=False)
# freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
    
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

#Fully connected blocks + Dropout
x = Dense(1024, activation='relu')(x)
x = Dropout(.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(.5)(x)
predictions = Dense(8, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)
model.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])



In [67]:
fname = 'models/inceptionV3_0/weights.{epoch:02d}-{val_loss:.2f}.h5'
callbacks = [ModelCheckpoint(path + fname,)]
model.fit_generator(tr_batches, tr_batches.nb_sample * 3, nb_epoch=5,
                    validation_data=val_batches, nb_val_samples=val_batches.nb_sample,
                    callbacks=callbacks)

Epoch 1/5
7893/7893 [==============================] - 300s - loss: 1.3482 - acc: 0.5387 - val_loss: 0.9973 - val_acc: 0.6457
Epoch 2/5
7893/7893 [==============================] - 291s - loss: 1.0939 - acc: 0.6138 - val_loss: 1.0388 - val_acc: 0.6134
Epoch 3/5
7893/7893 [==============================] - 293s - loss: 1.0049 - acc: 0.6439 - val_loss: 0.9274 - val_acc: 0.6518
Epoch 4/5
7893/7893 [==============================] - 293s - loss: 0.9209 - acc: 0.6727 - val_loss: 0.7965 - val_acc: 0.7443
Epoch 5/5
7893/7893 [==============================] - 290s - loss: 0.8872 - acc: 0.6752 - val_loss: 0.7500 - val_acc: 0.7347


## Generate Submission

In [68]:
from nc_utils import *

In [ ]:

# model.load_weights('../../data/nature_conservancy/weights/cnn_20170131_epoch_0.h5')
gen = image.ImageDataGenerator()
test_data = get_batches(path + 'test_stg1/', shuffle=False, 
                        batch_size=32, target_size=(299,299))

#Predict class probabilities
preds = model.predict_generator(test_data, test_data.nb_sample)
#Clip at 75% for scoring on kaggle
preds_clip = do_clip(preds,.75)


Found 1000 images belonging to 1 classes.


In [ ]:
img_ids = [img.split('/')[1] for img in test_data.filenames]
results, fname = create_submission_file(preds_clip, img_ids, "inceptionV3_0")
comp = 'the-nature-conservancy-fisheries-monitoring'
info = 'inceptionV3_0 + Data Aug.'
cmd = gen_submission_cmd(fname, comp, info):
print(cmd)